### 1) 라이브러리 설치

In [ ]:
# poetry add pypdf

Note: you may need to restart the kernel to use updated packages.


#### PyPDFLoader 간단한 예제

In [1]:
from dotenv import load_dotenv
import os

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_X


In [ ]:
import os
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# PDF 파일 경로 설정
pdf_filepath = '../data/tutorial-korean.pdf'

# 파일 존재 여부 확인 (파일이 없으면 오류 발생)
if not os.path.exists(pdf_filepath):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {pdf_filepath}")

try:
    # 1. PDF 파일 로드
    loader = PyPDFLoader(pdf_filepath)  # PDF 파일을 로드할 객체 생성
    docs = loader.load()  # 문서를 전체 로드

    # 총 문서 개수 출력
    print(f"총 {len(docs)}개의 문서가 로드 되었습니다.")

    #  첫 번째 문서의 메타데이터 출력
    print("첫 번째 문서 메타데이터:")
    print(json.dumps(docs[0].metadata, indent=2, ensure_ascii=False))

    # 특정 인덱스(10번째) 문서의 내용 확인 (존재할 경우)
    if len(docs) > 10:
        print("\n10번째 문서 내용:", type(docs[10]))
        print(docs[10])  # 10번째 문서 출력

    #  2. 텍스트 분할 (200자 단위, 중첩 없음)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
    split_docs = loader.load_and_split(text_splitter=text_splitter)  # 분할된 문서 로드

    # 분할된 문서 개수 출력
    print(f"\n분할된 문서의 개수: {len(split_docs)} 타입: {type(split_docs)}")

    # 10번째 분할된 문서 내용 출력 (존재할 경우)
    if len(split_docs) > 10:
        print("\n10번째 분할된 문서:")
        print(split_docs[10])

    # 3. Lazy Load 방식으로 문서 로드
    print("\nLazy Load 방식으로 문서 로드:")
    for i, doc in enumerate(loader.lazy_load()):
        if i < 5:  # 너무 많은 출력 방지 (예제: 처음 5개만 출력)
            print(json.dumps(doc.metadata, indent=2, ensure_ascii=False))

except Exception as e:
    # 오류 발생 시 메시지 출력
    print(f"오류 발생: {e}")


#### OllamaEmbeddings
* bge-m3:latest 모델

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_ollama import OllamaEmbeddings

print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader('../data/tutorial-korean.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 청크 크기 (한국어 최적화)
    chunk_overlap=200,      # 중복 부분 (맥락 보존)
    separators=["\n\n", "\n", ".", " ", ""] # 자연스러운 분할을 위한 구분자
)

chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")

print("==> 3. 벡터화 → 임베딩으로 변환")
# embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     dimensions=1536
# )

embeddings = OllamaEmbeddings(model="bge-m3:latest")

print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")
vectorstore.save_local("faiss_db")

print("===> 5. 검색 → 질문과 유사한 문서 찾기")
# VectorStoreRetriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}  # 상위 6개 관련 문서 검색
)
print(" Retriever 설정 완료")

print("===> 6. 생성 → LLM으로 답변 생성")
# llm = ChatOpenAI(
#     model="gpt-4o-mini",
#     temperature=0.1,
#     max_tokens=1500
# )
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

# 한국어 최적화 프롬프트
prompt_template = """
당신은 BlueJ 프로그래밍 환경 전문가입니다. 
아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 규칙:
1. 문서 내용만을 근거로 답변하세요
2. 단계별 설명이 필요하면 순서대로 작성하세요  
3. 구체적인 메뉴명, 버튼명을 포함하세요
4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요

답변:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print(" 프롬프트 설정 완료")

# ===================================
# 7. QA 체인 생성 Retriever + LLM을 체인으로 연결
# ===================================
print("\n ===> 7.  QA 체인 생성...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
print("  RAG 파이프라인 구축 완료!")

question = "BlueJ에서 객체를 생성하는 방법은 무엇인가요?"
result = qa_chain.invoke({"query": question})
print(result)

==> 1. 문서 로딩 → PDF 읽기...
  총 39페이지 로드 완료
==> 2. 문서 분할 → 작은 청크로 나누기
  66개 청크 생성 완료
  평균 청크 길이: 688자
==> 3. 벡터화 → 임베딩으로 변환
==> 4. 저장 → FAISS 벡터스토어에 저장
 FAISS 벡터스토어 생성 완료 (66개 벡터)
===> 5. 검색 → 질문과 유사한 문서 찾기
 Retriever 설정 완료
===> 6. 생성 → LLM으로 답변 생성
 프롬프트 설정 완료

 ===> 7.  QA 체인 생성...
  RAG 파이프라인 구축 완료!
{'query': 'BlueJ에서 객체를 생성하는 방법은 무엇인가요?', 'result': 'BlueJ에서 객체를 생성하는 방법은 다음과 같습니다.\n\n1. 메인 창 가운데에 있는 클래스 아이콘(예: Staff) 위에서 마우스 오른쪽 버튼을 클릭합니다.  \n2. 팝업 메뉴에서 원하는 생성자(constructor)를 선택합니다.  \n3. 나타난 대화상자에 객체 이름을 입력합니다. 기본 이름(예: staff_1)이 제시되며, 그대로 사용해도 무관합니다.  \n4. OK 버튼을 클릭하면 객체가 만들어지고 오브젝트 벤치(object bench)에 표시됩니다.  \n\n요약: 클래스 팝업 메뉴에서 생성자를 고른 뒤 대화상자에서 이름을 확인하고 OK를 누르면 객체가 생성됩니다.', 'source_documents': [Document(id='f568e4f8-09f9-493d-9916-365ab4579717', metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7D1B1DBC6A9C5E4B8AEBEF

In [3]:
from pprint import pprint

pprint(result)

{'query': 'BlueJ에서 객체를 생성하는 방법은 무엇인가요?',
 'result': 'BlueJ에서 객체를 생성하는 방법은 다음과 같습니다.\n'
           '\n'
           '1. 메인 창 가운데에 있는 클래스 아이콘(예: Staff) 위에서 마우스 오른쪽 버튼을 클릭합니다.  \n'
           '2. 팝업 메뉴에서 원하는 생성자(constructor)를 선택합니다.  \n'
           '3. 나타난 대화상자에 객체 이름을 입력합니다. 기본 이름(예: staff_1)이 제시되며, 그대로 사용해도 '
           '무관합니다.  \n'
           '4. OK 버튼을 클릭하면 객체가 만들어지고 오브젝트 벤치(object bench)에 표시됩니다.  \n'
           '\n'
           '요약: 클래스 팝업 메뉴에서 생성자를 고른 뒤 대화상자에서 이름을 확인하고 OK를 누르면 객체가 생성됩니다.',
 'source_documents': [Document(id='f568e4f8-09f9-493d-9916-365ab4579717', metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>', 'source': '../data/tutorial-korean.pdf', 'total_pages': 39, 'page': 9, 'page_label': '10'}, page_content='10이 대화상자는 생성할 객체의 이름을 입력받습니다. 또한 동시에 기본적인 이름(staff_

In [4]:

# ===================================
# 8. 테스트 질문들
# ===================================
test_questions = [
    "BlueJ에서 객체를 생성하는 방법은 무엇인가요?",
    "컴파일 오류가 발생했을 때 어떻게 확인할 수 있나요?", 
    "디버깅을 위해 중단점을 설정하는 방법을 알려주세요",
    "코드패드는 무엇이고 어떻게 사용하나요?",
    "애플릿을 만들고 실행하는 방법을 설명해주세요"
]

print("\n" + "=" * 60)
print(" RAG 시스템 테스트")
print("=" * 60)

# ===================================
# 9. 질문 및 답변 실행
# ===================================
for i, question in enumerate(test_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)


 RAG 시스템 테스트

【테스트 1/5】
 질문: BlueJ에서 객체를 생성하는 방법은 무엇인가요?
 답변 생성 중...

 답변:
--------------------------------------------------
BlueJ에서 객체를 생성하는 방법은 다음과 같습니다.

1. 클래스 아이콘 위에서 마우스 오른쪽 버튼을 클릭합니다.  
2. 팝업 메뉴에서 원하는 생성자(constructor)를 선택합니다.  
3. 나타난 대화상자에서 객체 이름을 입력합니다(기본 이름이 제시되며, 그대로 사용해도 무관합니다).  
4. OK 버튼을 클릭합니다.  

이렇게 하면 오브젝트 벤치(object bench)에 새로운 객체가 나타납니다.

 참조 문서:
   1. 페이지 9: 10이 대화상자는 생성할 객체의 이름을 입력받습니다. 또한 동시에 기본적인 이름(staff_1)이  제공됩니다. 이 이름(staff1)은 지금 ...
   2. 페이지 7: 83.2. 프로젝트 열기 프로젝트란, 관련 있는 여러개의 파일들을 묶어서 관리하기 위한 하나의 작업 단위를 말합니다.  BlueJ  프로젝트는 ...
   3. 페이지 35: 3610.6. 라이브러리 클래스로부터 객체생성하기 BlueJ 는 JDK에서 제공하는 라이브러리의 클래스들로부터 객체를 생성하기 위한 기능을  제...

----------------------------------------

【테스트 2/5】
 질문: 컴파일 오류가 발생했을 때 어떻게 확인할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
컴파일 오류 확인 방법

1. 에디터 정보창 확인  
   - 소스 코드에 오류를 만들고 Compile 버튼을 누르면, 에디터 하단의 정보창(information area)에 에러 메시지가 출력됩니다.  
   - 에러가 발생한 라인은 반전(하이라이트)되어 보입니다.

2. 도움말 보기  
   - 정보창 오른쪽에 나타나는